In [ ]:
import pandas as pd

In [ ]:
folder = "../run/data/Norfolk"
dir_save = "../run/data/stream_data"

In [ ]:
df_meta = pd.read_csv(folder + "/Norfolk_metadata.csv")
df_meta

In [ ]:
df_meta.sensor_filename.unique()

In [ ]:
json_scenario = {}

In [ ]:
json_scenario['1'] = {
    "name": "baseline",
    "date_start": "2006-01-01",
    "date_end": "2006-05-31"
}

json_scenario['2'] = {
    "name": "excessive_infiltration_0.4",
    "date_start": "2006-06-01",
    "date_end": "2006-06-30"
}

json_scenario['3'] = {
    "name": "baseline",
    "date_start": "2006-07-01",
    "date_end": "2006-12-31"
}

In [ ]:
df_syn = pd.DataFrame([])

print("############################")
for sce in json_scenario.keys():
    print(f"scenario number = {sce}")
    sce_name = json_scenario[sce]["name"]
    print(f"scenario name = {sce_name}")
    df_temp = pd.read_csv(folder + f"/{sce_name}.csv", index_col = 0)
    
    if sce_name != "baseline":
        sce_name = sce_name.rsplit("_",1)[0]
        print(f"parsed scenario name = {sce_name}")
    
    df_temp["label"] = sce_name
    print(f"data shape = {df_temp.shape}")
    df_temp.index = pd.to_datetime(df_temp.index)
    date_start = json_scenario[sce]['date_start']
    date_end = json_scenario[sce]['date_end']
    df_temp = df_temp[date_start:date_end]
    
    df_syn = pd.concat([df_syn, df_temp.reset_index()], ignore_index=True)
    
    print("############################")
    
df_syn = df_syn.set_index("OS_time")
df_syn.to_csv(dir_save + "/stream_data_preformatted.csv")
df_syn

In [ ]:
def get_timeinterval(os_timestamp):
    
        #converting timestamp to pandas datetime
        timestamp = pd.to_datetime(os_timestamp)
        # inferring timestep (frequency) from the dataframe
        dt = timestamp.diff().value_counts().idxmax() # in pandas timedelta
        dt = int(dt.value/(10**9)/60) # in minutes
        print("[Training/Testing Data Processing] timestep ({} min) inferred from the dataframe".format(dt))
        return dt

In [ ]:
temp_stream_FDD_data_test = pd.read_csv("C:/Users/jkim4/Documents/GitHub/FDD_RF_Repo/run/data/stream_data/stream_data_preformatted.csv")
timestep = get_timeinterval(temp_stream_FDD_data_test.iloc[:,0]) # in minutes
aggregate_n_runs = int(60/timestep*24)
df_label = temp_stream_FDD_data_test.set_index("OS_time").iloc[:,-1:]
df_label.index = pd.to_datetime(df_label.index)
resample_frequency = str(24) + "H"
df_label = df_label.resample(resample_frequency).bfill()
temp_stream_FDD_data_test = temp_stream_FDD_data_test.copy().iloc[:,0:-1]
temp_stream_FDD_data_test = temp_stream_FDD_data_test.groupby(temp_stream_FDD_data_test.index // (aggregate_n_runs)).mean().iloc[:,0:-8]
temp_stream_FDD_data_test['label'] = df_label.values
#temp_stream_FDD_data_test.to_csv(self.configs['dir_data_stream'] + f'\\{stream_data_file_name.replace("_preformatted","_formatted")}')
#fault_inputs_output_test = pd.concat([fault_inputs_output_test, temp_stream_FDD_data_test], axis = 0)

In [ ]:
temp_stream_FDD_data_test

In [ ]:
for col in temp_stream_FDD_data_test.columns:
    print(col)